# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [ ]:
import os
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [ ]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [ ]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [ ]:
song_files = get_files("data/song_data")
print(song_files)

In [ ]:
filepath = song_files[0]

In [ ]:
df = pd.read_json(filepath, lines=True)
df.head()

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [ ]:
song_data = 
song_data

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [ ]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [ ]:
artist_data = 
artist_data

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [ ]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [ ]:
log_files = 

In [ ]:
filepath = 

In [ ]:
df = 
df.head()

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [ ]:
df = 
df.head()

In [ ]:
t = 
t.head()

In [ ]:
time_data = ()
column_labels = ()

In [ ]:
time_df = 
time_df.head()

#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [ ]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [ ]:
user_df = 

#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [ ]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [ ]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = ()
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.

#### Upgrade Postgres Version

This will upgrade the Postgres version from 9.x to 13.x so that we can use newer features like generated columns and not have to convert things in application code that can be done automatically in the DB.

In [1]:
# Stop the postgres
!service postgresql stop
# Disable the service
!systemctl disable postgresql
# Install latest postgres
!sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
!wget --quiet -O - https://www.postgresql.org/media/keys/ACCC4CF8.asc | apt-key add -
!apt-get update
# In terminal, we don't need --allow-unathenticated, but we seem to in Jupyter
!apt-get install -y --allow-unauthenticated postgresql postgresql-contrib

 * Stopping PostgreSQL 9.5 database server
   ...done.
Synchronizing state of postgresql.service with SysV init with /lib/systemd/systemd-sysv-install...
Executing /lib/systemd/systemd-sysv-install disable postgresql
insserv: warning: current start runlevel(s) (empty) of script `postgresql' overrides LSB defaults (2 3 4 5).
insserv: warning: current stop runlevel(s) (0 1 2 3 4 5 6) of script `postgresql' overrides LSB defaults (0 1 6).
gpg: no valid OpenPGP data found.
Get:1 http://ppa.launchpad.net/ubuntu-toolchain-r/test/ubuntu xenial InRelease [23.8 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [109 kB]     
Get:3 http://apt.postgresql.org/pub/repos/apt xenial-pgdg InRelease [76.6 kB]  
Hit:4 http://archive.ubuntu.com/ubuntu xenial InRelease                        
Get:6 http://archive.ubuntu.com/ubuntu xenial-updates InRelease [109 kB]       
Hit:7 https://deb.nodesource.com/node_11.x xenial InRelease                    
Get:8 https://cli-assets.heroku.com/a

In [2]:
# List clusters
!pg_lsclusters
# Drop the auto-created cluster, now that we have the proper libraries installed, upgrade 9.5 to 13.
!pg_dropcluster 13 main
!pg_upgradecluster 9.5 main

Ver Cluster Port Status Owner    Data directory               Log file
9.5 main    5432 down   postgres /var/lib/postgresql/9.5/main /var/log/postgresql/postgresql-9.5-main.log
13  main    5433 down   postgres /var/lib/postgresql/13/main  /var/log/postgresql/postgresql-13-main.log
Restarting old cluster with restricted connections...
Creating new PostgreSQL cluster 13/main ...
/usr/lib/postgresql/13/bin/initdb -D /var/lib/postgresql/13/main --auth-local peer --auth-host md5 --encoding SQL_ASCII --lc-collate C --lc-ctype C
The files belonging to this database system will be owned by user "postgres".
This user must also own the server process.

The database cluster will be initialized with locales
  COLLATE:  C
  CTYPE:    C
  MESSAGES: en_US.UTF-8
  MONETARY: en_US.UTF-8
  NUMERIC:  en_US.UTF-8
  TIME:     en_US.UTF-8
The default text search configuration will be set to "english".

Data page checksums are disabled.

fixing permissions on existing directory /var/lib/postgresql/13/main ..

Now we drop the installed version and upgrade the existing one (so we can leverage the existing settings and don't have to redo users, passwords, permissions, etc).

In [3]:
# Check to make sure the new 13 was created from the 9.5, drop the old 9.5
!pg_lsclusters
!pg_dropcluster 9.5 main

Ver Cluster Port Status Owner    Data directory               Log file
9.5 main    5433 down   postgres /var/lib/postgresql/9.5/main /var/log/postgresql/postgresql-9.5-main.log
13  main    5432 down   postgres /var/lib/postgresql/13/main  /var/log/postgresql/postgresql-13-main.log


In [4]:
# This removes an extension for columnar analytics. It's outdated for postgres 13 and probably not necessary for this exercise.
# If we do want that support it's all part of the `citrus` extension now. This is a JIC. It may not exist in the new conf. If it does, and
# we don't comment it out, it will error.
!sed -i "s/shared_preload_libraries = 'cstore_fdw'/# shared_preload_libraries = 'cstore_fdw'/" /etc/postgresql/13/main/postgresql.conf
# Start the new cluster
!pg_ctlcluster 13 main start

Now make sure it all worked. The following should output:
```
psql (PostgreSQL) 13.X.X (Ubuntu 13.X-X.xxxxx.xx+x)
13/main (port 5432): online
```

In [5]:
# Check the version and the status
!psql -V
!service postgresql status

psql (PostgreSQL) 13.3 (Ubuntu 13.3-1.pgdg16.04+1)
13/main (port 5432): online


#### Create the tables for the ETL

Install extra dependencies:

- Pydash for functional-ish style
- Pendulum for a better DateTime interface

In [6]:
!pip install pydash
!pip install pendulum

    100% |████████████████████████████████| 92kB 3.6MB/s ta 0:00:011
    100% |████████████████████████████████| 163kB 6.8MB/s ta 0:00:01
    100% |████████████████████████████████| 491kB 16.7MB/s ta 0:00:01


Run the `create_tables` script to setup the database...

In [ ]:
%run create_tables.py

Run the `etl` script and load the data

In [ ]:
%run etl.py